# **Testing implementation of shift + scale fitting on Climate Explorer**

**Check results & plots obtained using Climate Explorer against those obtained using extRemes package**

_Always assume that 2023 is year of interest, with counterfactual covariate -0.93514 (so that mu0 and sigma0 are at GMST = 0)_

In [1]:
source("../methods_extRemes.r")

gmst <- read.fwf("ts/gmst.txt", comment.char = "#", widths = c(5,20), col.names = c("year", "gmst"))
gmst_2023 <- gmst$gmst[gmst$year == 2023]
gmst_cf <- 0

# method to extract parameter estimates from Climate Explorer output
cx_par_ests <- function(parstring) {
    cols <- gsub("\\.\\.\\.", "",unlist(strsplit(paste0(unlist(strsplit(cx[grepl(parstring, cx)], "(<.+?>)")), collapse = ""), " +")))
    setNames(as.numeric(cols[length(cols) - (2:0)]), c("bestimate", "lower", "upper"))[c(2,1,3)]
}

# method to modify size of plotting area
prep_window <- function(rc = c(1,1), w = 4, h = 4, ...) { options(repr.plot.width = rc[2]*w, repr.plot.height = rc[1]*h, repr.plot.res = 200); par(mfrow = rc, pch = 20, ...) }

# method to quickly transform observations to a given covariate
stransf <- function(mdl, qcov) {
    pars <- findpars(mdl, qcov = qcov)
    sort(revtrans.evd(trans(mdl), loc = sum(pars$loc), scale = sum(pars$scale), shape = pars$shape))
}

## **GEV - maximum temperatures**

In [2]:
dist <- "GEV"
data <- "TXx-deBilt"

# load time series & fit model using extRemes package
df <- merge(gmst, read.fwf(paste0("ts/",data,".txt"), comment.char = "#", widths = c(5,20), col.names = c("year", "var")))
mdl <- fevd(var, df, location.fun = ~gmst, scale.fun = ~gmst, type = dist)

# extract parameters from Climate Explorer output & R fitted model
cx <- readLines(paste0("cx/cx_",tolower(dist),"_",data,".txt")) 
cx_res <- t(sapply(c("mu0" = "mu;':</td><td>  -0.94", "mu1" = "alpha;:", "sigma0" = "sigma;':</td><td>  -0.94", "sigma1" = "beta;:", "shape" = "xi;:"), cx_par_ests))
r_res <- matrix(ci(mdl, type = "parameter"), dim(cx_res))

### **Parameter estimates**
_Update parameter selection according to model fitted_

In [3]:
# extract parameters from Climate Explorer output & R fitted model
cx <- readLines(paste0("cx/cx_",tolower(dist),"_",data,".txt")) 
cx_res <- t(sapply(c("mu0" = "mu;':</td><td>  -0.94", "mu1" = "alpha;:", "sigma0" = "sigma;':</td><td>  -0.94", "sigma1" = "beta;:", "shape" = "xi;:"), cx_par_ests))
r_res <- matrix(ci(mdl, type = "parameter"), dim(cx_res))

In [4]:
k <- nrow(cx_res) # number of parameters, to sort out plot size
prep_window(c(1,k / 2.5))

png(paste0("fig/", tolower(dist),"_",data,"_par-ests.png"), height = 360, width = 480 * 2); par(cex.main = 1.4); {
    par(mfrow = c(1,k), oma = c(0,0,2,0))
    for (i in 1:k) {
        plot(rep(1,3), cx_res[i,], type = "l", lwd = 5, col = adjustcolor("blue", 0.4), xlim = c(0.5, 2.5), ylim = range(pretty(c(cx_res[i,], r_res[i,]))),
             main = rownames(cx_res)[i], xlab = "", ylab = "", xaxt = "n")
        lines(rep(2,3), r_res[i,], type = "l", lwd = 5, col = adjustcolor("red", 0.4))
        points(1,cx_res[i,2], pch = 20, lwd = 5, col = "darkblue")
        points(2,r_res[i,2], pch = 20, lwd = 5, col = "darkred")
        axis(1, at = c(1,2), labels = c("CX", "R"))
    }
    mtext(paste0("Comparison of parameter estimates - ",dist,", ",data), outer = T, font = 2)
}; dev.off()

png 
  2

### **Transformed observations & return level plots**
_Convert fitted values to negative if necessary_

In [5]:
# read Climate Explorer output & split into factual/counterfactual
cx <- read.table(paste0("cx/cx_",tolower(dist),"_",data,".txt"), col.names = c("n", "gumbel", "Y", "fit", "rp", "date"), na.strings = "-999.900")

cx_cf <- cx[1:((nrow(cx)-2) / 2),]
cx_f <- cx[-c(1:((nrow(cx)-2) / 2),nrow(cx)-(0:1)),]
cx_obs <- cx[nrow(cx),"fit"]

# calculate the necessary return levels from model fitted in R (NB sign correction needed on fitted values)
qcov_f <- make.qcov(mdl, vals = list("mu1" = gmst_2023, "sigma1" = gmst_2023))
qcov_cf <- make.qcov(mdl, vals = list("mu1" = gmst_cf, "sigma1" = gmst_cf))

obs_f <- stransf(mdl, qcov = qcov_f)
obs_cf <- stransf(mdl, qcov = qcov_cf)

exp_cf <- return.level(mdl, cx_cf[-(1:20),c("rp")], qcov = qcov_cf)
exp_f <- return.level(mdl, cx_f[-(1:20),c("rp")], qcov = qcov_f)

In [6]:
# compare transformed observations

prep_window(c(1,2))

xylim <- range(pretty(c(exp_f, exp_cf, cx_cf$fit[-(1:20)], cx_f$fit[-(1:20)])))

png(paste0("fig/", tolower(dist),"_",data,"_plotted-values.png"), height = 480 * 1, width = 480 * 2); par(cex.main = 1.4); {
    par(mfrow = c(1,2), oma = c(0,0,2,0), pch = 20)

    plot(0, type = "n", xlim = xylim, ylim = xylim, xlab = "Calculated in R", ylab = "Calculated in Climate Explorer", main = "Transformed observations")
    points(obs_cf, cx_cf[cx_cf$date > 0,c("Y")], col = "blue")
    points(obs_f, cx_f[cx_f$date > 0,c("Y")], col = "firebrick")
    abline(0,1)

    plot(0, type = "n", xlim = xylim, ylim = xylim, xlab = "Calculated in R", ylab = "Calculated in Climate Explorer", main = "Expected values")
    points(exp_cf, cx_cf[-(1:20),c("fit")], col = "blue")
    points(exp_f, cx_f[-(1:20),c("fit")], col = "firebrick")
    abline(0,1)

    mtext(paste0("Comparison of plotted values - ",dist,", ",data), outer = T, font = 2)
}; dev.off()

png 
  2

In [7]:
## Return level plots

prep_window(c(1,2))
png(paste0("fig/", tolower(dist),"_",data,"_return_levels.png"), height = 480 * 1, width = 480 * 2); par(cex.main = 1.4); {
    par(mfrow = c(1,2), oma = c(0,0,2,0), pch = 20)

    plot(0, type = "n", xlim = range(cx_cf$rp), ylim = xylim, log = "x", xlab = "Return period (years)", ylab = data, main = "Climate Explorer")
        lines(cx_cf[-(1:20),c("rp", "fit")], col = "blue", lwd = 2)
        lines(cx_f[-(1:20),c("rp", "fit")], col = "firebrick", lwd = 2)

        points(cx_cf[cx_cf$date > 0,c("rp","Y")], col = "blue", pch = 4, lwd = 2, cex = 0.7)
        points(cx_f[cx_f$date > 0,c("rp","Y")], col = "firebrick", pch = 4, lwd = 2, cex = 0.7)

    plot(0, type = "n", xlim = range(cx_cf$rp), ylim = xylim, log = "x", xlab = "Return period (years)", ylab = data, main = "Calculated in R")
        lines(cx_cf[-(1:20),c("rp")], exp_cf, col = "blue", lwd = 2)
        lines(cx_f[-(1:20),c("rp")], exp_f, col = "firebrick", lwd = 2)

        points(cx_cf[cx_cf$date > 0,c("rp")], obs_cf, col = "blue", pch = 4, lwd = 2, cex = 0.7)
        points(cx_f[cx_f$date > 0,c("rp")], obs_f, col = "firebrick", pch = 4, lwd = 2, cex = 0.7)

    mtext(paste0("Return level plots - ",dist,", ",data), outer = T, font = 2)
}; dev.off()

png 
  2

### **GMST trend**
_Convert fitted values to negative if necessary_

In [12]:
# load fitted GMST trend from Climate Explorer
cx_fit <- read.table(paste0("cx/cx_",tolower(dist),"_",data,"_obsplot.txt"), comment.char = "#", skip = nrow(df)+5,
                    col.names = c("gmst", "var", "rl6", "rl40"))

## GMST trend plots
prep_window(c(1,2)); par(mfrow = c(1,1))
png(paste0("fig/", tolower(dist),"_",data,"_gmsttrend.png"), height = 480 * 1, width = 480 * 2); par(cex.main = 1.4); {
    plot(df[,c("gmst", "var")], pch = 20, ylim = range(pretty(c(unlist(cx_fit[,-1]), df$var))), xlab = "GMST", ylab = data,
        main = paste0("Fitted trend - ",dist,", ",data))
    matplot(cx_fit[1:101,1], cx_fit[1:101,-1], type = "l", lty = 1, add = T, col = "black", lwd = 3:1)

    lines(sort(df$gmst), findpars(mdl)$location[order(df$gmst)], col = "red", lty = 2, lwd = 3)
    matplot(sort(df$gmst), return.level(mdl, c(6,40))[order(df$gmst),], type = "l", lty = 2, add = T, col = "red", lwd = 2:1)
}; dev.off()

png 
  2

## **GEV - minimum temperatures**

In [29]:
dist <- "GEV"
data <- "TNn-deBilt"

# load time series & fit model using extRemes package
df <- merge(gmst, read.fwf(paste0("ts/",data,".txt"), comment.char = "#", widths = c(5,20), col.names = c("year", "var")))

# fit model to negative temperatures so that we can still look at block maxima
df$nvar <- -df$var
mdl <- fevd(nvar, df, location.fun = ~gmst, scale.fun = ~gmst, type = dist)

### **Parameter estimates**
_Update parameter selection according to model fitted_

In [30]:
# extract parameters from Climate Explorer output & R fitted model
cx <- readLines(paste0("cx/cx_",tolower(dist),"_",data,".txt")) 
cx_res <- t(sapply(c("mu0" = "mu;':</td><td>  -0.94", "mu1" = "alpha;:", "sigma0" = "sigma;':</td><td>  -0.94", "sigma1" = "beta;:", "shape" = "xi;:"), cx_par_ests))
r_res <- matrix(ci(mdl, type = "parameter"), dim(cx_res))

# correct signs of parameter estimates
cx_res["sigma0",] <- -cx_res["sigma0",]
cx_res["sigma1",] <- -cx_res["sigma1",]
r_res[1:2,] <- -r_res[1:2,]

In [30]:
k <- nrow(cx_res) # number of parameters, to sort out plot size
prep_window(c(1,k / 2.5))

png(paste0("fig/", tolower(dist),"_",data,"_par-ests.png"), height = 360, width = 480 * 2); par(cex.main = 1.4); {
    par(mfrow = c(1,k), oma = c(0,0,2,0))
    for (i in 1:k) {
        plot(rep(1,3), cx_res[i,], type = "l", lwd = 5, col = adjustcolor("blue", 0.4), xlim = c(0.5, 2.5), ylim = range(pretty(c(cx_res[i,], r_res[i,]))),
             main = rownames(cx_res)[i], xlab = "", ylab = "", xaxt = "n")
        lines(rep(2,3), r_res[i,], type = "l", lwd = 5, col = adjustcolor("red", 0.4))
        points(1,cx_res[i,2], pch = 20, lwd = 5, col = "darkblue")
        points(2,r_res[i,2], pch = 20, lwd = 5, col = "darkred")
        axis(1, at = c(1,2), labels = c("CX", "R"))
    }
    mtext(paste0("Comparison of parameter estimates - ",dist,", ",data), outer = T, font = 2)
}; dev.off()

png 
  2

### **Transformed observations & return level plots**
_Convert fitted values to negative if necessary_

In [31]:
# read Climate Explorer output & split into factual/counterfactual
cx <- read.table(paste0("cx/cx_",tolower(dist),"_",data,".txt"), col.names = c("n", "gumbel", "Y", "fit", "rp", "date"), na.strings = "-999.900")

cx_cf <- cx[1:((nrow(cx)-2) / 2),]
cx_f <- cx[-c(1:((nrow(cx)-2) / 2),nrow(cx)-(0:1)),]
cx_obs <- cx[nrow(cx),"fit"]

# calculate the necessary return levels from model fitted in R (NB sign correction needed on fitted values)
qcov_f <- make.qcov(mdl, vals = list("mu1" = gmst_2023, "sigma1" = gmst_2023))
qcov_cf <- make.qcov(mdl, vals = list("mu1" = gmst_cf, "sigma1" = gmst_cf))

obs_f <- -stransf(mdl, qcov = qcov_f)
obs_cf <- -stransf(mdl, qcov = qcov_cf)

exp_cf <- -return.level(mdl, cx_cf[-(1:20),c("rp")], qcov = qcov_cf)
exp_f <- -return.level(mdl, cx_f[-(1:20),c("rp")], qcov = qcov_f)

In [33]:
# compare transformed observations

prep_window(c(1,2))

xylim <- range(pretty(c(exp_f, exp_cf, cx_cf$fit[-(1:20)], cx_f$fit[-(1:20)])))

png(paste0("fig/", tolower(dist),"_",data,"_plotted-values.png"), height = 480 * 1, width = 480 * 2); par(cex.main = 1.4); {
    par(mfrow = c(1,2), oma = c(0,0,2,0), pch = 20)

    plot(0, type = "n", xlim = xylim, ylim = xylim, xlab = "Calculated in R", ylab = "Calculated in Climate Explorer", main = "Transformed observations")
    points(obs_cf, cx_cf[cx_cf$date > 0,c("Y")], col = "blue")
    points(obs_f, cx_f[cx_f$date > 0,c("Y")], col = "firebrick")
    abline(0,1)

    plot(0, type = "n", xlim = xylim, ylim = xylim, xlab = "Calculated in R", ylab = "Calculated in Climate Explorer", main = "Expected values")
    points(exp_cf, cx_cf[-(1:20),c("fit")], col = "blue")
    points(exp_f, cx_f[-(1:20),c("fit")], col = "firebrick")
    abline(0,1)

    mtext(paste0("Comparison of plotted values - ",dist,", ",data), outer = T, font = 2)
}; dev.off()

png 
  2

In [34]:
## Return level plots

prep_window(c(1,2)); par(mfrow = c(1,1))
png(paste0("fig/", tolower(dist),"_",data,"_return_levels.png"), height = 480 * 1, width = 480 * 2); par(cex.main = 1.4); {
    par(mfrow = c(1,2), oma = c(0,0,2,0), pch = 20)

    plot(0, type = "n", xlim = range(cx_cf$rp), ylim = xylim, log = "x", xlab = "Return period (years)", ylab = data, main = "Climate Explorer")
        lines(cx_cf[-(1:20),c("rp", "fit")], col = "blue", lwd = 2)
        lines(cx_f[-(1:20),c("rp", "fit")], col = "firebrick", lwd = 2)

        points(cx_cf[cx_cf$date > 0,c("rp","Y")], col = "blue", pch = 4, lwd = 2, cex = 0.7)
        points(cx_f[cx_f$date > 0,c("rp","Y")], col = "firebrick", pch = 4, lwd = 2, cex = 0.7)

    plot(0, type = "n", xlim = range(cx_cf$rp), ylim = xylim, log = "x", xlab = "Return period (years)", ylab = data, main = "Calculated in R")
        lines(cx_cf[-(1:20),c("rp")], exp_cf, col = "blue", lwd = 2)
        lines(cx_f[-(1:20),c("rp")], exp_f, col = "firebrick", lwd = 2)

        points(cx_cf[cx_cf$date > 0,c("rp")], obs_cf, col = "blue", pch = 4, lwd = 2, cex = 0.7)
        points(cx_f[cx_f$date > 0,c("rp")], obs_f, col = "firebrick", pch = 4, lwd = 2, cex = 0.7)

    mtext(paste0("Return level plots - ",dist,", ",data), outer = T, font = 2)
}; dev.off()

png 
  2

### **GMST trend**
_Convert fitted values to negative if necessary_

In [110]:
# load fitted GMST trend from Climate Explorer
cx_fit <- read.table(paste0("cx/cx_",tolower(dist),"_",data,"_obsplot.txt"), comment.char = "#", skip = nrow(df)+5,
                    col.names = c("gmst", "var", "rl6", "rl40"))

## GMST trend plots
prep_window(c(1,2))
png(paste0("fig/", tolower(dist),"_",data,"_gmsttrend.png"), height = 480 * 1, width = 480 * 2); par(cex.main = 1.4); {
    plot(df[,c("gmst", "var")], pch = 20, ylim = range(pretty(c(unlist(cx_fit[,-1]), df$var))), xlab = "GMST", ylab = data,
        main = paste0("Fitted trend - ",dist,", ",data))
    matplot(cx_fit[1:101,1], cx_fit[1:101,-1], type = "l", lty = 1, add = T, col = "black", lwd = 3:1)

    lines(sort(df$gmst), -findpars(mdl)$location[order(df$gmst)], col = "red", lty = 2, lwd = 3)
    matplot(sort(df$gmst), -return.level(mdl, c(6,40))[order(df$gmst),], type = "l", lty = 2, add = T, col = "red", lwd = 2:1)
}; dev.off()

# **Custom optimisation**

In [242]:
source("../../wwa_model-fitting.R")

# method to get a quick parametric confidence interval on parameters
par_ci_normal <- function(mdl, alpha = 0.05) {
    
    z.alpha <- qnorm(alpha/2, lower.tail = FALSE)
    
    p <- mdl$par
    
    cov.theta <- solve(mdl$hessian)
    var.theta <- diag(cov.theta)
    if (any(var.theta < 0)) 
        stop("ci: negative Std. Err. estimates obtained.  Not trusting any of them.")
    
    cbind("lower" = p - z.alpha * sqrt(var.theta), "est" = p, "upper" = p + z.alpha * sqrt(var.theta))
}

## **Test my code against FEVD**

In [243]:
dist <- "GEV"
data <- "TXx-deBilt"

# load time series & fit model
df <- merge(gmst, read.fwf(paste0("ts/",data,".txt"), comment.char = "#", widths = c(5,20), col.names = c("year", "var")))
mdl_fevd <- fevd(var, df, location.fun = ~gmst, scale.fun = ~gmst, type = dist, optim.args = list("method" = "BFGS"))
mdl_cb <- fit_ns(tolower(dist), "shiftscale", df, "var", "gmst", method = "BFGS")

In [244]:
# check that function gives same results against extRemes - looks good enough to use for Gaussian model fitting
mdl_fevd$results$value
mdl_cb$value

round(mdl_fevd$results$par - mdl_cb$par[c("mu0", "alpha", "sigma0", "beta", "shape")], 4)

[1] 247.9546

[1] 247.9546

mu0    mu1 sigma0 sigma1  shape 
 1e-04  4e-04  2e-04 -2e-04  0e+00

In [251]:
# check estimated confidence intervals
round(ci.fevd(mdl_fevd, type = "parameter")[c(1,3,2,4,5),] - par_ci_normal(mdl_cb), 4)

,95% lower CI,Estimate,95% upper CI
mu0,0e+00,1e-04,1e-04
sigma0,1e-04,2e-04,2e-04
mu1,4e-04,4e-04,4e-04
sigma1,-3e-04,-2e-04,-1e-04
shape,0e+00,0e+00,1e-04


## **Gaussian - mean temperatures**

In [252]:
dist <- "norm"
data <- "TGg-deBilt"

# load time series & fit model using extRemes package
df <- merge(gmst, read.fwf(paste0("ts/",data,".txt"), comment.char = "#", widths = c(5,20), col.names = c("year", "var")))
mdl <- fit_ns(tolower(dist), "shiftscale", df, "var", "gmst")

### **Parameter estimates**
_Update parameter selection according to model fitted_

In [253]:
# extract parameters from Climate Explorer output & R fitted model
cx <- readLines(paste0("cx/cx_",tolower(dist),"_",data,".txt")) 
cx_res <- t(sapply(c("mu0" = "mu;':</td><td> -999", "mu1" = "alpha;:", "sigma0" = "sigma;':</td><td> -999", "sigma1" = "beta;:"), cx_par_ests))
r_res <- par_ci_normal(mdl)[c("mu0","alpha","sigma0","beta"),]

In [254]:
k <- nrow(cx_res) # number of parameters, to sort out plot size
prep_window(c(1,k / 2.5))

png(paste0("fig/", tolower(dist),"_",data,"_par-ests.png"), height = 360, width = 480 * 2); par(cex.main = 1.4); {
    par(mfrow = c(1,k), oma = c(0,0,2,0))
    for (i in 1:k) {
        plot(rep(1,3), cx_res[i,], type = "l", lwd = 5, col = adjustcolor("blue", 0.4), xlim = c(0.5, 2.5), ylim = range(pretty(c(cx_res[i,], r_res[i,]))),
             main = rownames(cx_res)[i], xlab = "", ylab = "", xaxt = "n")
        lines(rep(2,3), r_res[i,], type = "l", lwd = 5, col = adjustcolor("red", 0.4))
        points(1,cx_res[i,2], pch = 20, lwd = 5, col = "darkblue")
        points(2,r_res[i,2], pch = 20, lwd = 5, col = "darkred")
        axis(1, at = c(1,2), labels = c("CX", "R"))
    }
    mtext(paste0("Comparison of parameter estimates - ",dist,", ",data), outer = T, font = 2)
}; dev.off()

png 
  2

### **Transformed observations & return level plots**
_Convert fitted values to negative if necessary_

In [255]:
# read Climate Explorer output & split into factual/counterfactual
cx <- read.table(paste0("cx/cx_",tolower(dist),"_",data,".txt"), col.names = c("n", "gumbel", "Y", "fit", "rp", "date"), na.strings = "-999.900")

cx_cf <- cx[1:((nrow(cx)-2) / 2),]
cx_f <- cx[-c(1:((nrow(cx)-2) / 2),nrow(cx)-(0:1)),]
cx_obs <- cx[nrow(cx),"fit"]

In [261]:
# calculate the necessary return levels from model fitted in R (NB sign correction needed on fitted values)
obs_f <- sort(stransform(mdl, fixed_cov = gmst_2023))
obs_cf <- sort(stransform(mdl, fixed_cov = gmst_cf))

exp_cf <- eff_return_level(cx_cf[-(1:20),c("rp")], mdl, fixed_cov = gmst_cf)
exp_f <- eff_return_level(cx_f[-(1:20),c("rp")], mdl, fixed_cov = gmst_2023)

In [262]:
# compare transformed observations

prep_window(c(1,2))

xylim <- range(pretty(c(exp_f, exp_cf, cx_cf$fit[-(1:20)], cx_f$fit[-(1:20)])))

png(paste0("fig/", tolower(dist),"_",data,"_plotted-values.png"), height = 480 * 1, width = 480 * 2); par(cex.main = 1.4); {
    par(mfrow = c(1,2), oma = c(0,0,2,0), pch = 20)

    plot(0, type = "n", xlim = xylim, ylim = xylim, xlab = "Calculated in R", ylab = "Calculated in Climate Explorer", main = "Transformed observations")
    points(obs_cf, cx_cf[cx_cf$date > 0,c("Y")], col = "blue")
    points(obs_f, cx_f[cx_f$date > 0,c("Y")], col = "firebrick")
    abline(0,1)

    plot(0, type = "n", xlim = xylim, ylim = xylim, xlab = "Calculated in R", ylab = "Calculated in Climate Explorer", main = "Expected values")
    points(exp_cf, cx_cf[-(1:20),c("fit")], col = "blue")
    points(exp_f, cx_f[-(1:20),c("fit")], col = "firebrick")
    abline(0,1)

    mtext(paste0("Comparison of plotted values - ",dist,", ",data), outer = T, font = 2)
}; dev.off()

png 
  2

In [267]:
## Return level plots

prep_window(c(1,2))
png(paste0("fig/", tolower(dist),"_",data,"_return_levels.png"), height = 480 * 1, width = 480 * 2); par(cex.main = 1.4); {
    par(mfrow = c(1,2), oma = c(0,0,2,0), pch = 20)

    plot(0, type = "n", xlim = range(cx_cf$rp), ylim = xylim, log = "x", xlab = "Return period (years)", ylab = data, main = "Climate Explorer")
        lines(cx_cf[-(1:20),c("rp", "fit")], col = "blue", lwd = 2)
        lines(cx_f[-(1:20),c("rp", "fit")], col = "firebrick", lwd = 2)

        points(cx_cf[cx_cf$date > 0,c("rp","Y")], col = "blue", pch = 4, lwd = 2, cex = 0.7)
        points(cx_f[cx_f$date > 0,c("rp","Y")], col = "firebrick", pch = 4, lwd = 2, cex = 0.7)

    plot(0, type = "n", xlim = range(cx_cf$rp), ylim = xylim, log = "x", xlab = "Return period (years)", ylab = data, main = "Calculated in R")
        lines(cx_cf[-(1:20),c("rp")], exp_cf, col = "blue", lwd = 2)
        lines(cx_f[-(1:20),c("rp")], exp_f, col = "firebrick", lwd = 2)

        points(cx_cf[cx_cf$date > 0,c("rp")], obs_cf, col = "blue", pch = 4, lwd = 2, cex = 0.7)
        points(cx_f[cx_f$date > 0,c("rp")], obs_f, col = "firebrick", pch = 4, lwd = 2, cex = 0.7)

    mtext(paste0("Return level plots - ",dist,", ",data), outer = T, font = 2)
}; dev.off()

png 
  2

### **GMST trend**
_Convert fitted values to negative if necessary_

In [266]:
# load fitted GMST trend from Climate Explorer
cx_fit <- read.table(paste0("cx/cx_",tolower(dist),"_",data,"_obsplot.txt"), comment.char = "#", skip = nrow(df)+5,
                    col.names = c("gmst", "var", "rl6", "rl40"))

## GMST trend plots
prep_window(c(1,2)); par(mfrow = c(1,1))
png(paste0("fig/", tolower(dist),"_",data,"_gmsttrend.png"), height = 480 * 1, width = 480 * 2); par(cex.main = 1.4); {
    plot(df[,c("gmst", "var")], pch = 20, ylim = range(pretty(c(unlist(cx_fit[,-1]), df$var))), xlab = "GMST", ylab = data,
        main = paste0("Fitted trend - ",dist,", ",data))
    matplot(cx_fit[1:101,1], cx_fit[1:101,-1], type = "l", lty = 1, add = T, col = "black", lwd = 3:1)

    lines(sort(df$gmst), ns_pars(mdl)$loc[order(df$gmst)], col = "red", lty = 2, lwd = 3)
    matplot(sort(df$gmst), eff_return_level(c(6,40), mdl)[order(df$gmst),], type = "l", lty = 2, add = T, col = "red", lwd = 2:1)
}; dev.off()

png 
  2